In [1]:
from sklearn.neighbors import NearestNeighbors

import pandas as pd

import numpy as np

import os

os.chdir('..')

In [2]:
data_path = r'data/dummy-data.csv'

queries_path = r'data/dummy-queries.csv'

In [3]:
data = pd.read_csv(data_path).astype(float)

In [4]:
queries = pd.read_csv(queries_path).astype(float)

In [5]:
def get_subset(data, query_type, v, l, r):
        
    if query_type == 1.0:

        return data[data['C'] == v]

    elif query_type == 2.0:

        return data[(data['T'] >= l) & (data['T'] <= r)]

    elif query_type == 3.0:

        return data[(data['C'] == v) & ((data['T'] >= l) & (data['T'] <= r))]
    
    return data

In [6]:
def conditional_brute_force_knn_search(data, queries):

    k = 100

    results = pd.DataFrame(columns = ['query_vector', 'Distance', 'data_vector', 'query_type', 'C', 'v', 'l', 'T', 'r'])

    for index, row in queries.iterrows():

        result_index = 0 + index * k

        query_type = row['query_type']

        v = row['v']

        l = row['l']

        r = row['r']

        query_id = row['vector']

        query_vector = row.filter(like='f').values

        subset = get_subset(data, query_type, v, l, r)
        
        find_neighbor_model = NearestNeighbors(n_neighbors=k, algorithm='brute')

        find_neighbor_model.fit(subset.filter(like='f').values)

        distance, data_indexes = find_neighbor_model.kneighbors(query_vector.reshape(1, -1))

        loop = 0

        for d_i in data_indexes.flatten():
            
            results.loc[result_index + loop, ['data_vector']] = data.loc[d_i, 'vector']

            C = data.loc[d_i, 'C']

            T = data.loc[d_i, 'T']

            results.loc[result_index + loop, 'query_vector'] = query_id

            results.loc[result_index + loop, 'Distance'] = distance.flatten()[loop]

            results.loc[result_index + loop, ['query_type', 'C', 'v', 'l', 'T', 'r']] = (query_type, C, v, l, T, r)

            loop += 1

    return results

In [7]:
res = conditional_brute_force_knn_search(data, queries)

res.to_parquet('output/BRUTE_FORCE.parquet')